# Gaze Analyser
This notebook takes in three files taken from the Pupil Recordings you have exported in Pupil Player. These are:
- `gaze_positions.csv` (contains raw data in regards to the gaze made throughout the recording)
- `info.player.json` (contains system and sync time used to format the recording timestamps)
- `fixations.csv` (contains events of where fixations has occured throughout the experiment)
- `annotations.csv` (contains annotations created in the experiment used for object interception/spawn, regions of observation, and when the experiment has begin and ended)

In [4]:
# Imports
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import warnings
import shutil


matplotlib.use('TkAgg')
warnings.filterwarnings("ignore", category=RuntimeWarning)

# File Paths
info_player_filePath = 'experiment_source/info.player.json'
gaze_csv_filePath = 'experiment_source/gaze_positions.csv'
annotations_filepath = 'experiment_source/annotations.csv'
fixation_filepath = 'experiment_source/fixations.csv'

# Contrasting Colours
CONTRASTING_COLURS = ['#011627', '#2ec4b6', '#e71d36', '#ff9f1c']
current_colour_index = 0

## Creating the DataFrame from the data

Reading the `info.player` JSON to retrieve `start_time_synced_s` and `start_time_system_s`. This is used to format the timestamp correctly to indicate time throughout the experiment:

In [5]:
def obtain_offset(filepath):
    with open(info_player_filePath, 'r') as file:
        data = json.load(file)  
    return data.get('start_time_system_s') - data.get('start_time_synced_s')

Now, reading the gaze_position.csv file to obtain base data, formating time, calculating smoothed positions, and angular distance and velocity of each timestamp. And removing data that contains infinte or NaN values.

In [6]:
# Calculate angular distance given Cartesian coordinates (x, y)
def calculate_angular_distance(x, y):
    return np.arctan2(y, x)

# Calculate velocity given angular distance and corresponding timestamps.
def calculate_velocity(angular_distance, timestamp):
    time_diff = np.diff(timestamp)
    angular_distance_diff = np.diff(angular_distance)
    velocity = angular_distance_diff / time_diff
    return np.concatenate(([np.nan], velocity))


# Applies Rolling Medium over the input field, and exports that into the output field
def smooth_data(df, input, output, window_size=0):
    df[output] = df[input].rolling(window=window_size).median()
    return df

def obtain_gaze_data(filepath: str, window=10, offset=0):
    csv_df = pd.read_csv(filepath)
    results_df = pd.DataFrame(columns=['gaze_timestamp',
                                       'time',
                                       'norm_pos_x', 
                                       'norm_pos_y', 
                                       'angular_distance',
                                       'angular_velocity',
                                       'movement_type',
                                       'smoothed_norm_pos_x', 
                                       'smoothed_norm_pos_y',
                                       'smoothed_angular_distance',
                                       'smoothed_angular_velocity',
                                       'smoothed_movement_type'])

    # Copy the data from csv of necessary fields
    results_df['gaze_timestamp'] = csv_df['gaze_timestamp']
    results_df['norm_pos_x'] = csv_df['norm_pos_x']
    results_df['norm_pos_y'] = csv_df['norm_pos_y']
    results_df['movement_type'] = 'None'
    results_df['smoothed_movement_type'] = 'None'
    
    # Populating the time field by adding the offset and then subtracting the minimum time to start from 0
    results_df['time'] = results_df['gaze_timestamp'] + offset
    results_df['time'] -= results_df['time'].min()

    # Using rolling mean to smooth the data and to rmeove as many extreme outliers that has been missed by Pupil Export
    results_df['smoothed_norm_pos_x'] = results_df['norm_pos_x'].rolling(window).median()
    results_df['smoothed_norm_pos_y'] = results_df['norm_pos_y'].rolling(window).median()

    # Calculating the angular distance for each x and y position for both smoothed and un-smoothed
    results_df['angular_distance'] = calculate_angular_distance(results_df['norm_pos_x'], results_df['norm_pos_y'])
    results_df['smoothed_angular_distance'] = calculate_angular_distance(results_df['smoothed_norm_pos_x'], results_df['smoothed_norm_pos_y'])

    # Calculating the angular velocity for both smooth and un-smoothed angular distances over time
    results_df['angular_velocity'] = calculate_velocity(results_df['angular_distance'], results_df['time'])
    results_df['smoothed_angular_velocity'] = calculate_velocity(results_df['smoothed_angular_distance'], results_df['time'])

    # Dropping NaN records and the `gaze_timestamp` field
    results_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    results_df.dropna(inplace=True)
    results_df = results_df.drop('gaze_timestamp', axis=1) # Remove the `gaze_timestamp` as that is not needed anymore

    return results_df

window_size=10
offset = obtain_offset(info_player_filePath)
gaze_df = obtain_gaze_data(gaze_csv_filePath, window=window_size, offset=offset)
print(f'There is a total of {len(gaze_df)} gaze positions')


There is a total of 8825 gaze positions


### Predicting Smooth Pursuit
Smooth pursuit occurs when the eyes tracks an moving object. There is roughly a constant velocity as the angular distance changes slowly, whereas Saccades are almost instant. We choose regions of the line where the change in velocity is under a threshold, and the change of angular distance is small enough to indicate the eyes moving between positions slowly.

In [7]:
def predict_smooth_pursuit(df, threshold_velocity=0.2, angular_distance_threshold=2e-4):
    pursuit_regions_velocity = np.abs(np.gradient(df['angular_velocity'])) < threshold_velocity
    pursuit_regions_distance = np.abs(np.gradient(df['angular_distance'])) > angular_distance_threshold
    pursuit_regions = (pursuit_regions_velocity) & (pursuit_regions_distance)
    df['movement_type'] = np.where((df['movement_type'] == 'None') & pursuit_regions, 'Smooth Pursuit', df['movement_type'])
    
    pursuit_regions_smoothed_velocity = np.abs(np.gradient(df['smoothed_angular_velocity'])) < threshold_velocity
    pursuit_regions_smoothed_distance = np.abs(np.gradient(df['smoothed_angular_distance'])) > angular_distance_threshold
    pursuit_regions_smoothed = (pursuit_regions_smoothed_velocity) & (pursuit_regions_smoothed_distance)
    df['smoothed_movement_type'] = np.where((df['smoothed_movement_type'] == 'None') & pursuit_regions_smoothed, 'Smooth Pursuit', df['smoothed_movement_type'])

predict_smooth_pursuit(gaze_df, threshold_velocity=0.2, angular_distance_threshold=2e-4)
smooth_pursuit = gaze_df[(gaze_df['movement_type'] == 'Smooth Pursuit') | (gaze_df['smoothed_movement_type'] == 'Smooth Pursuit')]
print(f'Total Smooth Pursuit Movements: {len(smooth_pursuit)}')

Total Smooth Pursuit Movements: 5405


### Predicting Saccades
Based on the research from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1190820/pdf/jphysiol00502-0164.pdf, Saccadic Movements have a stereotypical velocity graph where large peaks indicates a fast movement between two points (saccades). These are almost like straight lines between points. We can say that if the velocity between two  points exceeds a threshold, then this is most possibly a saccadic movement.

In [8]:
def predict_saccades(df, threshold_velocity=0.6):
    saccade_regions_high = df['angular_velocity'] > threshold_velocity
    accade_regions_low = df['angular_velocity'] < -threshold_velocity
    saccade_regions = saccade_regions_high | accade_regions_low
    df['movement_type'] = np.where(saccade_regions, 'Saccades', 'None')
    
    saccade_regions_high = df['smoothed_angular_velocity'] > threshold_velocity
    accade_regions_low = df['smoothed_angular_velocity'] < -threshold_velocity
    saccade_regions = saccade_regions_high | accade_regions_low
    df['smoothed_movement_type'] = np.where((df['smoothed_movement_type'].isin(['None', 'Smooth Pursuit'])) & saccade_regions, 'Saccades', df['smoothed_movement_type'])

predict_saccades(gaze_df, threshold_velocity=0.6)
saccades = gaze_df[(gaze_df['movement_type'] == 'Saccades') | (gaze_df['smoothed_movement_type'] == 'Saccades')]
print(f'Total Saccadic Movements: {len(saccades)}')

Total Saccadic Movements: 4760


### Predicting Fixation
Fixations are defined as gaze remain fixed around a stationary point. It is not directly fixed but the user gazes around the same point for a long period of time.

Pupil Player provides another csv file called `fixations` which identifies the timestamp of when a fixation occurs, and how long it can take. The function `plot_fixation_graph` takes regions starting from whatever second the fixation starts up until the end of the fixation indicated by by the `duration` field.

In [9]:
def predict_fixation(df, offset, time_offset=0, filepath='fixations.csv'):
    fixations_df = pd.read_csv(filepath)
    fixations_df['time'] = fixations_df['start_timestamp'] + offset
    fixations_df['time'] -= fixations_df['time'].min()

    for _, fixation in fixations_df.iterrows():
        start_time = fixation['time']
        duration = (fixation['duration'] / 1000.0) - time_offset  # Convert into seconds

        # Mark rows within the fixation region as 'Fixation' in both columns
        fixation_regions = (df['time'] >= start_time) & (df['time'] <= start_time + duration)
        df.loc[fixation_regions, 'movement_type'] = 'Fixation'
        df.loc[fixation_regions, 'smoothed_movement_type'] = 'Fixation'

predict_fixation(gaze_df, offset, time_offset=0.1, filepath=fixation_filepath)
fixation = gaze_df[(gaze_df['movement_type'] == 'Fixation') | (gaze_df['smoothed_movement_type'] == 'Fixation')]
print(f'Total Fixation Movements: {len(fixation)}')

Total Fixation Movements: 1182


## Incorporating Annotations
Using the annotation.csv file, we want to generate new csv files that contains the following: object information, regions of observation, and experiment start/end

In [10]:
def analyse_annotations(annotations_csv_filepath, offset, fill_threshold=1):
    # This works through the the annotation and finds the following:
    # What object each data point is looking at
    # When an object spawns and intercepted

    # Group every spawn and interception into a single record (not all spawn may have an interception), could group by object ID?
    annotations_df = pd.read_csv(annotations_csv_filepath)
    
    # Format the timestamp into time relative to the experiment
    annotations_df['time'] = annotations_df['timestamp'] + offset
    annotations_df['time'] -= annotations_df['time'].min()

    # Filter only the data that contains 'Intercepted' or 'Spawning'.
    # Each object has their own spawning and interception time, could create records of these two
    # We can utilise this data as our own way to simplify the visualising and also finding TTC

    # Filter rows with 'Intercepted' or 'Spawning' label
    object_annotations = annotations_df[annotations_df['label'].isin(['Intercepted', 'Spawning'])]
    # objects_df = object_annotations.pivot_table(index='id', columns='label', values=['time', 'ObjectSpeed'])
    objects_df = object_annotations.pivot_table(index='id', columns='label', values=['time', 'ObjectSpeed'])
    objects_df.reset_index(inplace=True)


    temp_df = pd.DataFrame(columns=['id',
                                       'ObjectSpeed',
                                       'Spawning', 
                                       'Intercepted'])
    
    temp_df['id'] = objects_df['id']
    temp_df['ObjectSpeed'] = objects_df['ObjectSpeed']['Spawning']
    temp_df['Spawning'] = objects_df['time']['Spawning']
    temp_df['Intercepted'] = objects_df['time']['Intercepted']

    objects_df = temp_df

    print(objects_df)

    # If any object does not have both 'Spawning' and 'Intercepted' events, fill NaN with appropriate values
    objects_df.fillna({'spawning_timestamp': pd.NaT, 'intercepted_timestamp': pd.NaT}, inplace=True)

    # Now, create a new dataframe which stores when a user is looking at a specific region
    # Obtains all records with the 'Looking At' label
    looking_at_df = annotations_df[annotations_df['label'].isin(['Looking At'])]

    # Initialise an empty DataFrame to store regions
    regions_df = pd.DataFrame(columns=['id', 'start_time', 'end_time'])

    # Initialise variables for tracking consecutive points
    current_obj_id = None
    start_time = None

    # Iterate through each row in the 'Looking At' dataframe
    for _, row in looking_at_df.iterrows():
        obj_id = row['id']
        timestamp = row['time']

        # Check if it's the same object and within the threshold seconds
        if obj_id == current_obj_id and start_time is not None and timestamp - start_time <= fill_threshold: 
            end_time = timestamp # Update the end time for the current region
        else:
            # A new region is created when the next ID does not match the current region's ID OR if the time between two points is greater than the threshold (indicating it's a new region for the same ID)
            if current_obj_id is not None and start_time is not None:
                regions_df = pd.concat([regions_df, pd.DataFrame({'id': [current_obj_id], 'start_time': [start_time], 'end_time': [end_time]})], ignore_index=True)

            # Update tracking variables for the next iteration
            current_obj_id = obj_id
            start_time = timestamp
            end_time = timestamp

    # Add the last region after the loop
    if current_obj_id is not None and start_time is not None:
        regions_df = pd.concat([regions_df, pd.DataFrame({'id': [current_obj_id], 'start_time': [start_time], 'end_time': [end_time]})], ignore_index=True)

    # May want to include the Experiment Start and End lines as a JSON
    start_time = annotations_df.loc[(annotations_df['label'] == 'Experiment Started'), 'time'].values
    end_time = annotations_df.loc[(annotations_df['label'] == 'Experiment Ended'), 'time'].values

    experiment_info = {
        'Start': start_time.item(),
        'End': end_time.item()
    }

    return objects_df, regions_df, experiment_info

objects_df, regions_df, experiment_info = analyse_annotations(annotations_filepath, offset)

      id  ObjectSpeed   Spawning  Intercepted
0    1.0         16.0   4.002075     4.664917
1    2.0         17.0   4.813354     5.466919
2    3.0         16.0   5.620667     6.324890
3    4.0         13.0   6.436890     7.206177
4    5.0         15.0   7.254150     7.939941
5    6.0         16.0   8.044800     8.718872
6    7.0         10.0   8.849304     9.803223
7    8.0         13.0   9.654602          NaN
8    9.0         19.0  10.460022    11.043396
9   10.0         16.0  11.275635    11.956421
10  11.0         14.0  12.071411    12.856934
11  12.0         12.0  12.876221    13.706482
12  13.0         12.0  13.693726    14.584839
13  14.0         15.0  14.508911    15.197998
14  15.0         17.0  15.320129    15.948914
15  16.0         12.0  16.133423    16.998047
16  17.0         10.0  16.937134    17.967773
17  18.0         13.0  17.750122          NaN
18  19.0         15.0  18.556091    19.252319
19  20.0         11.0  19.358887    20.232727
20  21.0         19.0  20.172974  

C:\Users\jhund\AppData\Local\Temp\ipykernel_7896\4227698401.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  regions_df = pd.concat([regions_df, pd.DataFrame({'id': [current_obj_id], 'start_time': [start_time], 'end_time': [end_time]})], ignore_index=True)


### Finding TTC per interception
This function only works if we have incorporated the annotation as we need the interception that has been made. This iterates through each interceptiom find all the points that falls below the time that interception has been made and take the most recent saccadic movement to calculate the Time-To-Contact by taking the difference between


In [11]:
def find_TTC(gaze_df, objects_df):
    results = []

    for index, row in objects_df.iterrows():
        # Extract the 'Intercepted' time
        intercepted_time = row['Intercepted']
        object_speed = row['ObjectSpeed']
        # Filter gaze_df based on the condition where 'time' is less than the intercepted_time
        filtered_gaze_df = gaze_df[gaze_df['time'] < intercepted_time]

        # Obtain all the records where the movement type is Saccades
        smoothed_saccadic_gaze_df = filtered_gaze_df[filtered_gaze_df['smoothed_movement_type'] == "Saccades"]
        saccadic_gaze_df = filtered_gaze_df[filtered_gaze_df['movement_type'] == "Saccades"]

        # Obtains the latest saccadic movement before the intercepted time
        recent_saccade = saccadic_gaze_df['time'].max()
        smoothed_recent_saccade = smoothed_saccadic_gaze_df['time'].max()


        # Calculate the saccadic movement before the interception was made which is the difference between latest saccade and interception
        time_to_contact = intercepted_time - recent_saccade
        smooth_time_to_contact = intercepted_time - smoothed_recent_saccade

        results.append({
            'Object ID': index+1, 
            'Object Speed': object_speed,
            'Interception': intercepted_time, 
            'Recent Saccade': recent_saccade, 
            'Time to Contact': time_to_contact,
            'Smoothed Recent Saccade': smoothed_recent_saccade,
            'Smoothed Time to Contact': smooth_time_to_contact})

    results_df = pd.DataFrame(results)
    return results_df

ttc_df = find_TTC(gaze_df, objects_df)
print(ttc_df)

    Object ID  Object Speed  Interception  Recent Saccade  Time to Contact  \
0           1          16.0      4.664917        4.655951         0.008966   
1           2          17.0      5.466919        5.451065         0.015854   
2           3          16.0      6.324890        6.310744         0.014147   
3           4          13.0      7.206177        7.190778         0.015399   
4           5          15.0      7.939941        7.934390         0.005552   
5           6          16.0      8.718872        8.718657         0.000215   
6           7          10.0      9.803223        9.802535         0.000688   
7           8          13.0           NaN             NaN              NaN   
8           9          19.0     11.043396       11.039724         0.003672   
9          10          16.0     11.956421       11.950490         0.005931   
10         11          14.0     12.856934       12.854383         0.002550   
11         12          12.0     13.706482       13.702416       

## Exporting Data
To be used for later use, this exports the dataframe and an json containing the offset for annotations

In [12]:
# Allows the dataframe to be exported as a csv file of a given name as well as a JSON file that stores the offset if the graph was to be used again
def export_gaze_data(gaze, objects, regions, ttc, experiment_info):
    
    output_directories = ['analysed_output']

    for output_directory in output_directories:
        if os.path.exists(output_directory):
            shutil.rmtree(output_directory)
        os.makedirs(output_directory)

    # Saves a copy into the analysed output
    gaze.to_csv(f'analysed_output/gaze.csv', index=False)
    regions.to_csv(f'analysed_output/regions.csv', index=False)
    ttc.to_csv(f'analysed_output/ttc.csv', index=False)
    objects.to_csv(f'analysed_output/objects.csv', index=False)
    with open('analysed_output/experiment_info.json', 'w') as json_file:
        json.dump(experiment_info, json_file)


    # Object is not correctly formatted to be exported, should fix this first:

export_gaze_data(gaze_df, objects_df, regions_df, ttc_df, experiment_info) # Exporting the dataframe as a csv file

# Gaze Visualiser
This part of the notebook takes in the exported data from the gaze analyser and displays them as an interactive graph to closely interpret the data. This uses the following files:
- `gaze.csv`
- `objects.csv`
- `regions.csv`
- `experiment_info.json`

In [13]:
# Imports
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
import warnings

matplotlib.use('TkAgg')
warnings.filterwarnings("ignore", category=RuntimeWarning)

# File Paths
regions_filepath = 'analysed_output/regions.csv'
gaze_filepath = 'analysed_output/gaze.csv'
objects_filepath = 'analysed_output/objects.csv'
experiment_info_filepath = 'analysed_output/experiment_info.json'


# Contrasting Colours
CONTRASTING_COLURS = ['#011627', '#2ec4b6', '#e71d36', '#ff9f1c']
current_colour_index = 0

Reading the CSV files as dataframes, and the experiment information a dictionary:

In [14]:
def open_files(gaze, objects, regions, ttc, experiment_info):
    gaze_df = pd.read_csv(gaze)
    objects_df = pd.read_csv(objects)
    regions_df = pd.read_csv(regions)
    ttc_df = pd.read_csv(ttc)
    with open(experiment_info, 'r') as file:
        data = json.load(file)
        exp_info = {
            'Start': data.get('Start'),
            'End': data.get('End')
        }

    return gaze_df, objects_df, regions_df, ttc_df, exp_info

gaze_df, objects_df, regions_df, experiment_info

print(objects_df)

gaze, objects, regions, exp_info, ttc_df = gaze_df, objects_df, regions_df, experiment_info, ttc_df

      id  ObjectSpeed   Spawning  Intercepted
0    1.0         16.0   4.002075     4.664917
1    2.0         17.0   4.813354     5.466919
2    3.0         16.0   5.620667     6.324890
3    4.0         13.0   6.436890     7.206177
4    5.0         15.0   7.254150     7.939941
5    6.0         16.0   8.044800     8.718872
6    7.0         10.0   8.849304     9.803223
7    8.0         13.0   9.654602          NaN
8    9.0         19.0  10.460022    11.043396
9   10.0         16.0  11.275635    11.956421
10  11.0         14.0  12.071411    12.856934
11  12.0         12.0  12.876221    13.706482
12  13.0         12.0  13.693726    14.584839
13  14.0         15.0  14.508911    15.197998
14  15.0         17.0  15.320129    15.948914
15  16.0         12.0  16.133423    16.998047
16  17.0         10.0  16.937134    17.967773
17  18.0         13.0  17.750122          NaN
18  19.0         15.0  18.556091    19.252319
19  20.0         11.0  19.358887    20.232727
20  21.0         19.0  20.172974  

## Plot TTC
Takes the object speed, and plots the TTC values 

In [15]:
def plot_ttc(ttc_df):
    figure, ax = plt.subplots(figsize=(16, 8))
    ax.scatter(ttc_df['Object Speed'], ttc_df['Time to Contact'])
    ax.set_title('Object Speed vs Time to Contact')
    ax.set_xlabel('Object Speed')
    ax.set_ylabel('Time to Contact')
    ax.grid(True)
    figure.show()

# plot_ttc(ttc_df)

## Displaying Gaze Positions
This is to visualise the difference smoothed and unsmoothed version of `norm_pos_x` and  `norm_pos_y`:

In [16]:
def plot_norm_positions(df):
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))
    axs[0].plot(df['time'], df['norm_pos_x'], label='Original norm_pos_x')
    axs[0].plot(df['time'], df['smoothed_norm_pos_x'], label=f'Smoothed norm_pos_x (window={10})', linestyle='--')
    axs[0].set_title('Norm_pos_x')

    axs[1].plot(df['time'], df['norm_pos_y'], label='Original norm_pos_y')
    axs[1].plot(df['time'], df['smoothed_norm_pos_y'], label=f'Smoothed norm_pos_y (window={10})', linestyle='--')
    axs[1].set_title('Norm_pos_y')

    for ax in axs:
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Position')
        ax.legend()
    fig.show()

# plot_norm_positions(gaze_df)

## Plotting the interception graph
The following functions looks at taking each of the dataframe, and plotting the graph according to using smoothed or unsmoothed positions.

### Angular Distance agaisnt Time Graph
The first graph plots the main graph that acts as the basis representation for gaze positions:

In [17]:
def plot_angular_distance(fig, ax, df, use_smooth=True, show_both=False):

    if show_both:
        ax.plot(df['time'], df['angular_distance'], label='Gaze Angular Distance',)
        ax.plot(df['time'], df['smoothed_angular_distance'], label=f'Angular Distance', linestyle='--')
    else:
        if use_smooth:
            ax.plot(df['time'], df['smoothed_angular_distance'], label=f'Angular Distance', color='orange')
        else:
            ax.plot(df['time'], df['angular_distance'], label='Gaze Angular Distance', color='orange')

    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Angular Distance (rad)', color='black')
    ax.legend(loc='lower right')
    fig.suptitle('Interception Experiment')
    fig.tight_layout()

# plot_angular_distance(figure, ax, gaze)

## Visualising Eye Movements
Within the dataframe, there is a field called `movement_type` where we identify which one of the three eye movements (Saccade, Fixations, and Smooth Pursuit) base on velocities and provided csv file from Pupil Lab.

### Regions of Saccades

In [18]:
def plot_saccades(ax, df, use_smooth=False):
    if use_smooth:
        source_distance = 'smoothed_angular_distance'
        movement_type= 'smoothed_movement_type'
    else:
        source_distance = 'angular_distance'
        movement_type = 'movement_type'

    distances = df[source_distance]
    saccade_regions = df[movement_type] == 'Saccades'

    ax.plot(df['time'], np.where(saccade_regions, distances, np.nan), color='red', label='Saccade Line')
    ax.legend(loc='lower right')

### Regions of Smooth Pursuit

In [19]:
def plot_smooth_pursuits(ax, df, use_smooth=False):
    if use_smooth:
        source_distance = 'smoothed_angular_distance'
        movement_type_column = 'smoothed_movement_type'
    else:
        source_distance = 'angular_distance'
        movement_type_column = 'movement_type'

    distances = df[source_distance]
    smooth_pursuit_regions = df[movement_type_column] == 'Smooth Pursuit'

    # Plot for Smooth Pursuit Data
    ax.plot(df['time'], np.where(smooth_pursuit_regions, distances, np.nan), color='green', label='Smooth Pursuit')
    ax.legend(loc='upper left')

### Regions of Fixation

In [20]:
def plot_fixations(ax, df, use_smooth=False):
    if use_smooth:
        source_distance = 'smoothed_angular_distance'
        movement_type= 'smoothed_movement_type'
    else:
        source_distance = 'angular_distance'
        movement_type = 'movement_type'

    distances = df[source_distance]
    fixation_regions = df[movement_type] == 'Fixation'

    ax.plot(df['time'], np.where(fixation_regions, distances, np.nan), color='blue', label='Fixation')
    ax.legend(loc='lower right')

### Combining the identification function:
This would combine each of the function and allow the user control on what they wish to present on the graph

In [21]:
def plot_data(figure, ax, gaze_df, use_smooth=True, fixations=False, smooth_pursuits=False, saccades=False):
    plot_angular_distance(figure, ax, gaze_df, use_smooth=use_smooth)
    if fixations:
        plot_fixations(ax, gaze_df, use_smooth=use_smooth)
    
    if smooth_pursuits:
        plot_smooth_pursuits(ax, gaze_df, use_smooth=use_smooth)

    if saccades:
        plot_saccades(ax, gaze_df, use_smooth=use_smooth)

    return figure, ax

### Visualising annotations
Annotations consists of multiple types. These are: Spawning, Intercepted, Looking At, Experiment End/Start. To simplify this approach, the analyser exports two files `regions.csv` and `objects.csv` which already has information which lets us display the regions and object information on the graph:

#### Adjusting and showing experiment start/end
Due to the recording starting before the experiment starts and delays before ending the experiment, we want to indicate which part of the graph is most relevant at first

In [22]:
def plot_experiment_lines(ax, experiment_info):
    start = experiment_info['Start']
    end = experiment_info['End']
    ax.axvline(x=start, color='black', linestyle='--')
    ax.axvline(x=end, color='black', linestyle='--')
    ax.set_xlim(start, end) # Adjust the graph to display between the lines when the graph appears

#### Representing Objects
To represent objects, each has been assigned a HEX code for a colour. They have been chosen as adjacent colours are contrasting for easy visualisation and avoid generating colours that might be close to each other:


In [23]:
def choose_colour():
    global current_colour_index
    colour = CONTRASTING_COLURS[current_colour_index] # Gets the colour in the current index
    current_colour_index = (current_colour_index + 1) % len(CONTRASTING_COLURS) # Increments the current index
    return colour

In [24]:
def assign_colour(objects_df):
    object_colours = {}
    for _, row in objects_df.iterrows():
        object_id = int(row['id'])
        object_colours[object_id] = choose_colour()
        # print(f'{object_id} with colour {object_colours[object_id]}')
    return object_colours

# object_colours = assign_colour(objects)

Once each object has been given a colour, we can start plotting the regions of observation, and spawn/intercept lines

In [25]:
def plot_objectLines(ax, objects_df, object_colours):
    for _, row in objects_df.iterrows():

        object_id = int(row['id'])
        line_colour = object_colours[object_id]

        spawn_time = row['Spawning']
        ax.axvline(x=spawn_time, color=line_colour, linestyle='--', alpha=0.7)
        ax.text(spawn_time, ax.get_ylim()[0] + 0.02, f'Spawned {object_id} at {spawn_time:.2f}', rotation=90, va='bottom', ha='right', color='black')

        intercepted_time = row['Intercepted']
        if intercepted_time != pd.NaT:
            ax.axvline(x=intercepted_time, color=line_colour, linestyle='--', alpha=0.7)
            ax.text(intercepted_time, ax.get_ylim()[1] - 0.02, f'Intercepted {object_id} at {intercepted_time:.2f}', rotation=90, va='top', ha='right', color='black')

# plot_objectLines(ax, objects, object_colours)

Plotting the observation across the experiment as regions on the graph

In [26]:
def plot_observations(ax, regions_df, object_colours):
    for _, row in regions_df.iterrows():
        object_id = row['id']
        colour = object_colours[object_id]
        start = row['start_time']
        end = row['end_time']

        ax.axvspan(start, end, color=colour, alpha=0.2)

# plot_observations(ax, regions, object_colours)

#### Combined Annotation functions
This function combines all the annotation related files

In [27]:
def plot_annotations(ax, regions_df, objects_df, experiment_info):
    object_colours = assign_colour(objects_df)
    plot_experiment_lines(ax, experiment_info)
    plot_objectLines(ax, objects_df, object_colours)
    plot_observations(ax, regions_df, object_colours)

## Displaying the graph

In [28]:
figure, ax = plt.subplots(figsize=(16, 8))
plot_data(figure, ax, gaze, use_smooth=True, fixations=True, smooth_pursuits=True, saccades=True)
# plot_ttc(ttc_df)
plot_annotations(ax, regions, objects, exp_info)

figure.show()